<a href="https://colab.research.google.com/github/Shehbaz-Niazi/Langchain/blob/main/RAG_mySelf_with_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Libraries**

In [20]:
!pip install -q -U langchain
!pip install -q -U langchain-google-genai
%pip install -qU langchain-pinecone

In [21]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [22]:
import time

index_name = "langchain-rag-with-pdf"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

# **PDF File Load and Connected**

In [24]:
!pip install langchain pymupdf

In [34]:
from google.colab import files
import fitz  # PyMuPDF

# 📌 User se PDF upload karwana
uploaded = files.upload()

# 📌 Uploaded file ka naam lena
pdf_filename = list(uploaded.keys())[0]
print(f"✅ File Uploaded: {pdf_filename}")


Saving champions trophy 2025.pdf to champions trophy 2025.pdf
✅ File Uploaded: champions trophy 2025.pdf


In [ ]:
def extract_text_from_pdf(pdf_filename):
    doc = fitz.open(pdf_filename)  # PDF Open karo
    text = ""

    for page_num, page in enumerate(doc):
        text += f"\n\n📌 Page {page_num + 1}:\n"  # Page number add karo
        text += page.get_text("text")  # Structured text extract karo

    return text

# 📌 Extract text
extracted_text = extract_text_from_pdf(pdf_filename)

# 📌 Pehle 1000 characters print karo
print(extracted_text[:1500])


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

pdf_embedding  = embeddings.embed_query(extracted_text)
print(pdf_embedding[:5])

# **Text Splitting**

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [46]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Har chunk ka size
    chunk_overlap=50  # Overlap taake context na lost ho
)

chunks = text_splitter.split_text(extracted_text)

In [47]:
documents = [Document(page_content=chunk, metadata={"source": pdf_filename}) for chunk in chunks]


In [48]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=pdf_embedding)

In [53]:
from pinecone import Pinecone, ServerlessSpec
import time
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document

# Vector Store ko initialize karna
vector_store = PineconeVectorStore(index=index, embedding=embeddings)




In [54]:
# Unique UUID generate karna

from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['38099d45-78a1-442b-b89b-71d18126f9b2',
 '346becc8-7c27-4a6f-b8cd-f6a0670d0bd2',
 'd9fd08a2-8006-4cf7-8f6e-ae8f6edd8ee3']

In [55]:
# Generative AI model
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


In [56]:
def answer_to_user(query:str):

  # vector search
  vector_results = vector_store.similarity_search(query, k=2)
  print(len(vector_results))

  # pass the model vector Result + User query
  final_answer = llm.invoke(f"Answer This User Query: {query}, Here are some refrences to answer: {vector_results}")

  return final_answer

In [57]:
answer =  answer_to_user("Tell me about important all rounder player in  icc champion trophy 2025 for Pakistan")
print(answer.content)

2
Based on the provided text, Shehbaz Khan Niazi is identified as an important all-rounder player for the Pakistan team in the ICC Champions Trophy 2025.
